# Boltz API Inference

This notebook demonstrates using the in-memory `boltz.api` interface to run structure predictions without relying on the command line or temporary files.

## Setup
Load required libraries, canonical molecules, and the model checkpoint.

In [ ]:
from pathlib import Path
from boltz.api import BoltzModel, ChainSpec, msa_from_a3m
from boltz.data.mol import load_canonicals

# Paths to the RDKit molecule database and pretrained checkpoint
moldir = Path('/path/to/molecules')
checkpoint = Path('/path/to/weights.ckpt')

# Load reference molecules and initialise the model
canonicals = load_canonicals(str(moldir))
model = BoltzModel(checkpoint, canonicals=canonicals, affinity=True)


## Protein–Protein prediction
Define two protein chains and run a structure prediction entirely in memory.

In [ ]:
from pathlib import Path

chain_a = ChainSpec(kind='protein', id='A', sequence='MAHHHHHHVAVDAVSFTLLQDQLQSVLDTLSEREAGVVRLRFGLTDGQPRTLDEIGQVYGVTRERIRQIESKTMSKLRHPSRSQVLRDYLDGSSGSGTPEERLLRAIFGEKA')
chain_b = ChainSpec(kind='protein', id='B', sequence='MRYAFAAEATTCNAFWRNVDMTVTALYEVPLGVCTQDPDRWTTTPDDEAKTLCRACPRRWLCARDAVESAGAEGLWAGVVIPESGRARAFALGQLRSLAERNGYPVRDHRVSAQSA')

# Optional: provide MSAs for each chain
msas = {
    'A': msa_from_a3m(Path('examples/msa/seq1.a3m').read_text()),
    'B': msa_from_a3m(Path('examples/msa/seq2.a3m').read_text()),
}

pp_result = model.predict_from_chains('prot-prot', [chain_a, chain_b], msas=msas)
# The prediction result exposes a StructureV2 instance and raw arrays
pp_result.structure, pp_result.raw.keys()


## Protein–Ligand prediction with affinity
Combine a protein chain with a ligand specified via SMILES and request a binding affinity estimate.

In [ ]:
protein_seq = (
    'MVTPEGNVSLVDESLLVGVTDEDRAVRSAHQFYERLIGLWAPAVMEAAHELGVFAALAEAPADSGELARRLDCDARAMRVLLDALYAYDVIDRIHDTNGFRYLLSAEARECL'
    'LPGTLFSLVGKFMHDINVAWPAWRNLAEVVRHGARDTSGAESPNGIAQEDYESLVGGINFWAPPIVTTLSRKLRASGRSGDATASVLDVGCGTGLYSQLLLREFPRWTATGLDVERIATLANAQALRL'
    'GVEERFATRAGDFWRGGWGTGYDLVLFANIFHLQTPASAVRLMRHAAACLAPDGLVAVVDQIVDADREPKTPQDRFALLFAASMTNTGGGDAYTFQEYEEWFTAAGLQRIETLDTPMHRILLARRATE'
    'PSAVPEGQASENLYFQ'
)
ligand = ChainSpec(kind='ligand', id='B', smiles='N[C@@H](Cc1ccc(O)cc1)C(=O)O')

chains = [ChainSpec(kind='protein', id='A', sequence=protein_seq), ligand]

pl_result = model.predict_from_chains('prot-ligand', chains, affinity_binder='B')
# Access the predicted binding affinity if the model was initialised with affinity=True
pl_result.affinity
